## Load Verification Dataset

In [ ]:
from clustering import LabelGuidedKMeansUtils
from importlib import reload
import clustering
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
n_outputs = 5
verification_csv = '../data/v3.2.2_train.csv'
X_orig, Y_orig = LabelGuidedKMeansUtils.load_dataset(verification_csv, n_outputs)

In [ ]:
df = pd.read_csv(verification_csv)
df.head(5)

In [ ]:
X, Y = X_orig.copy(), Y_orig.copy()
X, Y = LabelGuidedKMeansUtils.remove_outliers(X, Y,tolerance= 10)

print(f'dropped {X_orig.shape[0] - X.shape[0]} outliers from {X_orig.shape[0]} inputs')
X.shape, Y.shape

## Generate Label Guided K-Means Regions

In [ ]:
num_iters = 3

In [ ]:
from clustering import LabelGuidedKMeans, LabelGuidedKMeansUtils

init = 150  #Initial number of clusters for k-means. Verified experimentally through elbow method.

for iters in range(num_iters):
    
    #init_centroid dictates the mechanism which the algorithm should follow for initial centroid generation.
    #The options are 'rand','mean','first','k-means++'. Check clustering.py for further details.
    
    lgkmc = LabelGuidedKMeans().fit(X, Y,init_nclusters = init, init_centroid='mean')
    
    LabelGuidedKMeansUtils.print_summary(lgkmc)
    
    init = None


In [ ]:
# # Data from the printed observation for visualization
# # reg = [['n == 1','n > 1','n >= 10','n >= 100','n >= 1000'], [5917,15175,5512,557,2]]
# # no_cen = [9147,18612,6034,445,1]
# # mean_weighted_k = [9064,18455,6027,455,3]
# x = np.arange(len(reg[0]))
# width = 0.2
# fig, ax = plt.subplots(figsize=(10,12))
# plt.yscale('log')
# rects1 = ax.bar(x-width, reg[1],width,label = 'Mean Centroid Initialization')
# rects2 = ax.bar(x, no_cen,width,label = 'Unmodified Label Guided K-means')
# rects3 = ax.bar(x+width, mean_weighted_k,width,label = 'Mean Aggregated weighted K-means')

# ax.set_ylabel('Number of regions')
# ax.set_title('Clustering results')
# ax.set_xticks(x)
# ax.set_xticklabels(reg[0])
# ax.legend()

# def autolabel(rects):
#     """Attach a text label above each bar in *rects*, displaying its height."""
#     for rect in rects:
#         height = rect.get_height()
#         ax.annotate('{}'.format(height),
#                     xy=(rect.get_x() + rect.get_width() / 3, height),
#                     xytext=(0, 3),  # 3 points vertical offset
#                     textcoords="offset points",
#                     ha='center', va='bottom',rotation = 'vertical')

# autolabel(rects1)
# autolabel(rects2)
# autolabel(rects3)

# plt.show()

In [ ]:
#Run this cell to be able to execute any changes made in clustering.py without having to restart the kernel.
reload(clustering)
LabelGuidedKMeans = clustering.LabelGuidedKMeans
LabelGuidedKMeansUtils = clustering.LabelGuidedKMeansUtils

In [ ]:
# save regions
LabelGuidedKMeansUtils.save(lgkmc, outpath='../logs/test/test_regions/lgkm.p')

## Verify Regions

In [ ]:
import os
from clustering import LabelGuidedKMeansUtils

# load the regions
lgkmc = LabelGuidedKMeansUtils.load('../logs/test/test_regions/lgkm.p')
# lgkmc = LabelGuidedKMeansUtils.load('/home/sai/Desktop/Marabou/scad_tot/verification/lgkmc.p')
regions = lgkmc.get_regions(sort=True)

In [ ]:
import os, sys
sys.path.append(os.path.abspath('../.marabou'))
from verification.utils import remove_softmax_activation
from verification.robustness import verify_regions
from verification.tot_net import AllowedMisclassifications

# Create verification model without softmax layer
model_path = '../network/models/latest/model-verification'
remove_softmax_activation('../network/models/latest/model.h5', save_path=model_path)

# verify the regions (they will be automatically saved to out_dir)
# note: you'll need the Gurobi Optimizer installed (https://github.com/NeuralNetworkVerification/Marabou#use-lp-relaxation)

out_dir = './results/cluster_verification/'
regions = verify_regions(
    network_path=model_path,
    network_version=1,
    network_options=dict(modelType='savedModel_v2'),
    marabou_options=dict(solveWithMILP=True, milpTightening='none'),
    regions=regions,
    e_min=0.0001,
    e_max=10.0,
    e_interval=0.00001,
    radius_padding=0,
    timeout=0,
    # marabou_verbosity=0,
    allowed_misclassifications=None,
    out_dir=out_dir
    )